In [1]:

url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
#change this size parameter to fetch product requests default(200)
size = 200

#code starts from here
import requests
from bs4 import BeautifulSoup
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os, time
import pandas as pd

C:\Users\rohit\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# code for part 1

In [2]:
BASE_DIR = os.path.dirname("__file__")
DRIVER_PATH = os.path.join(BASE_DIR,'chromedriver.exe')
JSON_FILE_PATH = os.path.join(BASE_DIR,'temp.json')

In [3]:
#initializing driver
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
serv_obj = Service(DRIVER_PATH)
driver = webdriver.Chrome(options = options,service = serv_obj)
driver.get(url)
driver.maximize_window()

#initializing wait object
mywait = WebDriverWait(driver,10)

In [4]:

def find_all_products_urls(all_a_tags):
    all_product_urls = []
    for a_tag in all_a_tags:
        all_product_urls.append(a_tag.get_attribute('href'))
    return all_product_urls

def find_all_product_names(all_a_tags):
    all_product_names = []
    for a_tag in all_a_tags:
        span_tag = a_tag.find_element(By.TAG_NAME,"span")
        name = span_tag.text
        all_product_names.append(name)
    return all_product_names  

def find_all_product_ratings(all_span_tags):
    all_product_ratings = []
    for span_tag in all_span_tags:
        rating = span_tag.get_attribute('innerHTML')
        all_product_ratings.append(rating)
    return all_product_ratings


def find_all_product_reviews(all_span_tags):
    all_product_reviews = []
    for span_tag in all_span_tags:
        review = span_tag.get_attribute('innerHTML').strip("()")
        all_product_reviews.append(review)
    return all_product_reviews

def find_all_product_prices(all_price_div_tags):
    all_prices = []
    for div_tag in all_price_div_tags:
        try:
            price_range_tag = div_tag.find_elements(By.CLASS_NAME,"a-price-range")
            price_tag = div_tag.find_element(By.CLASS_NAME,"a-price")
        except Exception as e:
#             print("price_range_not_exists")
            continue
        if len(price_range_tag) > 0:
            price_range_tag = price_range_tag[0]
            price_span_tag = price_range_tag.find_elements(By.CLASS_NAME,"a-price-whole")[0]
        else:
            price_span_tag = price_tag.find_element(By.CLASS_NAME,"a-price-whole")
        price = price_span_tag.get_attribute('innerHTML')
        all_prices.append(price)
    return all_prices

In [5]:
def partFirstScraper():
    all_product_names = []
    all_product_urls = []
    all_product_ratings = []
    all_product_reviews = []
    mywait.until(EC.presence_of_all_elements_located((By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"]/a')))

    #finding all a tags which contains product links and names
    all_a_tags = driver.find_elements(By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"]/a')

    #finding all span tags which contains product ratings
    # mywait.until(EC.presence_of_all_elements_located((By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="a-section a-spacing-none a-spacing-top-micro"]//*[@class="a-row a-size-small"]//')))
    all_ratings_span_tags = driver.find_elements(By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="a-section a-spacing-none a-spacing-top-micro"]//*[@class="a-row a-size-small"]//*[@class="a-icon-alt"]')
    all_reviews_span_tags = driver.find_elements(By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="a-section a-spacing-none a-spacing-top-micro"]//*[@class="a-row a-size-small"]//*[@class="a-size-base s-underline-text"]')
    all_price_div_tags = driver.find_elements(By.XPATH,'//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-small a-spacing-top-small"]//*[@class="sg-row"]//*[@class="sg-col-inner"]//*[@class="a-section a-spacing-none a-spacing-top-micro s-price-instructions-style"]')



    all_product_names = find_all_product_names(all_a_tags)
    all_product_urls = find_all_products_urls(all_a_tags)
    all_product_ratings = find_all_product_ratings(all_ratings_span_tags)
    all_product_reviews = find_all_product_reviews(all_reviews_span_tags)
    all_prices = find_all_product_prices(all_price_div_tags)

    return {'Product URL':all_product_urls,'Product Name':all_product_names,'Product Price':all_prices,'Rating':all_product_ratings,'Number of Reviews':all_product_reviews}

In [6]:
#fetching all page listings       
data_dict = {
            "Product Name":[],
            "Product URL":[],
            "Product Price":[],
            "Rating":[],
            "Number of Reviews":[]
}
while True:
    try:
        mywait.until(EC.presence_of_all_elements_located((By.LINK_TEXT,"Next")))
        next_button = driver.find_element(By.LINK_TEXT,"Next")
        
        #Part 1 Logic here
        part_first_dict = partFirstScraper()
        data_dict["Product Name"] = data_dict["Product Name"] + part_first_dict["Product Name"]
        data_dict["Product URL"] = data_dict["Product URL"] + part_first_dict["Product URL"]
        data_dict["Product Price"] = data_dict["Product Price"] + part_first_dict["Product Price"]
        data_dict["Rating"] = data_dict["Rating"] + part_first_dict["Rating"]
        data_dict["Number of Reviews"] = data_dict["Number of Reviews"] + part_first_dict["Number of Reviews"]
        
        next_button.click()
        
    except TimeoutException as e:
        break

driver.quit()

part 1 Output

In [14]:
dataframe = pd.DataFrame({'Product Name':data_dict['Product Name'][:300],'Product URL':data_dict["Product URL"][:300],'Product Price':data_dict["Product Price"][:300],'Rating': data_dict["Rating"][:300],'Number of Reviews':data_dict['Number of Reviews'][:300]})
dataframe.to_csv("part1.csv")
dataframe

,Product Name,Product URL,Product Price,Rating,Number of Reviews
0,"URBAN TRIBE Havana 15.6"" inch Laptop Backpack ...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,998,3.9 out of 5 stars,684
1,Amazon Basics - Laptop Backpack/Casual Backpac...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"1,209",1.0 out of 5 stars,1
2,Wesley Milestone 2.0 Casual Waterproof Laptop ...,https://www.amazon.in/Wesley-Milestone-Waterpr...,565,4.3 out of 5 stars,"9,226"
3,American Tourister 32 Ltrs Black Casual Backpa...,https://www.amazon.in/American-Tourister-AMT-S...,"1,299",4.1 out of 5 stars,"51,370"
4,ADISA 15.6 inch Laptop Backpack Office Bag Col...,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,499,3.9 out of 5 stars,462
...,...,...,...,...,...
295,Amazon Brand - Eden & Ivy Women's Sling Bag,https://www.amazon.in/Amazon-Brand-Womens-Slin...,"558<span class=""a-price-decimal"">.</span>",4.4 out of 5 stars,"6,048"
296,Wenger SYNERGY 16 Inch Laptop Backpack with Ta...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"6,440",3.8 out of 5 stars,"1,033"
297,uppercase SealPro Professional Laptop Backpack...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"1,900",5.0 out of 5 stars,1
298,"Nivia 5227 Pulse Junior Bag, Others (Orange) (...",https://www.amazon.in/Nivia-5227-Junior-Others...,369,1.0 out of 5 stars,1


# Code for part 2

In [8]:
class WebPageDownloader:
    def __init__(self):
        self.headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
    def getAllProductUrlPages(self,all_product_url_list):
        all_pages_data =[]
        for url in all_product_url_list:
            response = requests.get(url,headers=self.headers)
            all_pages_data.append(response.text)
        return all_pages_data

In [9]:

def getDescription(dom):
    desc_list = [desc.text for desc in dom.xpath('//*[@id="feature-bullets"]//li//span')]
    return desc_list

def getProductDescription(dom):
    product_description = ''
    data = [x.text for x in dom.xpath('//*[@class="aplus-v2 desktop celwidget"]//div//*')]
    data = [x.strip(" \n") for x in data if x is not None and x != ' ' and x != '\n']
    for line in data:
        line = line.split("(function(f)")[0]
        product_description += str(line)
    return product_description

def getTechnicalDetails(dom):
    with_table_data_th = dom.xpath('//*[@id="productDetails_techSpec_section_1"]//th')
    with_table_data_td = dom.xpath('//*[@id="productDetails_techSpec_section_1"]//td')
    without_table_data = dom.xpath('//*[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]//span[@class="a-list-item"]/span')


    if without_table_data != []:
        details = []
        without_table_data = [data.text for data in without_table_data]

        for data in without_table_data:
            details.append(data.split("\n")[0].strip(" \n"))

        headers = []
        desc = []

        for i,det in enumerate(details):
            if i%2 == 0:
                headers.append(det)
            else:
                desc.append(det)
        technical_details = dict(zip(headers,desc))
    else:
        with_table_data_th = [data.text for data in with_table_data_th]
        with_table_data_th = [data.strip(" \n") for data in with_table_data_th]

        with_table_data_td = [data.text for data in with_table_data_td]
        with_table_data_td = [data.strip(" \n") for data in with_table_data_td]


        technical_details = dict(zip(with_table_data_th,with_table_data_td))
    return technical_details


In [10]:

def scraper(data):
    soup = BeautifulSoup(data,'lxml')
    dom = etree.HTML(str(soup))

    description = getDescription(dom)
    product_description = getProductDescription(dom)
    technical_details = getTechnicalDetails(dom)

    return {'description':description,'product_description':product_description,'technical_details':technical_details}


In [11]:
#part 2 code here
all_product_urls = data_dict['Product URL'][:size]
downloader = WebPageDownloader()
all_pages_data = downloader.getAllProductUrlPages(all_product_urls)
final_data = []
asin_list = []
manufacturer_list = []
for data in all_pages_data:
    final_data.append(scraper(data))
for data in final_data:
    tech_det = data['technical_details']
    asin_list.append(tech_det.get('ASIN'))
    manufacturer_list.append(tech_det.get('Manufacturer'))

In [12]:
frame_data = {'description':[],'Product Description':[],'ASIN':[],'Manufacturer':[]}
for data in final_data:
    frame_data['description'].append(data['description'])
    frame_data['Product Description'].append(data['product_description'])
frame_data['ASIN'] = asin_list
frame_data['Manufacturer'] = manufacturer_list

df = pd.DataFrame(frame_data)
df

,description,Product Description,ASIN,Manufacturer
0,"[ Laptop Compatibility: Yes, Laptop size: 15.6...",Read moreErgonomically DesignedThe Laptop Ba...,B01LXNNFDF,"Backpack International Pvt Ltd, Backpack Inter..."
1,"[ Laptop backpack made from water-resistant, s...",,None,‎Flab internatioanl private limited
2,[ 30L Capacity: The Backpack has a padded lapt...,"Ergonomic Contoured ShapeWith a slim, sleek an...",None,‎Wesley
3,"[ Laptop Compatibility: No, Strap Type: Adjust...",Read moreRead moreRead moreRead moreRead more,None,‎Samsonite
4,[ Material: Water Resistant Light-Weight Polye...,DESCRIPTIONLight weight and spacious laptop ba...,B09TPX22NF,"ADISA, ADISA INTERNATIONAL, 42/34, Upper Groun..."
...,...,...,...,...
195,[ One zippered compartment suitable for keys/n...,,B0862LF7HS,"F Gear, Uber Fashion Merchandise Pvt. Ltd. No...."
196,[ Compartment: Outside 4 compartments includin...,"Red LemonPremium Quality , Affordable RatesPre...",B082MKHDJ1,None
197,"[ EUME's Made in India Laptop Backpack, made o...",,B07GLTYX1C,"EUME, Gala No 6 , Ground Floor, Bldg No.193,In..."
198,"[ Fit Type: Regular , Triple Protect high-de...",Wenger retained its distinct identity after th...,B004RG25BK,"Wenger, +91-22-62628888"


In [13]:
#exporting to csv
df.to_csv("part2.csv")